In [2]:
import numpy as np
import pandas as pd
import math

## Read this cell before your work on this exam.
1. Do not import any other packages. You only can use three packages imported in the first cell.
2. Due time of this exam is 2pm on 10/31/2017. Any submission after this time is not acceptable. Please submit your .ipynb file for question 1-3 through Canvas on time.
3. Please write all necessary steps and final answer of question 4 and brint it to me in next Tuesday's class.
4. Please work on this exam by yourself. Do not discuss and share anything with other students. Email me if you have any questions. 

## Question1 (25 pts)
### Please refer lab3 to answer this question.
For the following vectors, x and y, calculate the indicated similarity or distance measures. Print out the results.

(a) x = (1,1,1,1), y = (2,2,2,2)  L1 distance, L2 distance

(b) x = (2,-1,0,2,0,-3), y = (-1,1,-1,0,0,-1) Linfinity distance, correlation coefficient

(c) x = (0,-1,0,1), y = (1,0,-1,0) correlation coefficient

In [3]:
# Your code goes here
from numpy import linalg as LA

a1, a2 = np.array([1,1,1,1]), np.array([2,2,2,2])
b1, b2 = np.array([2,-1,0,2,0,-3]), np.array([-1,1,-1,0,0,-1])
c1, c2 = np.array([0,-1,0,1]), np.array([1,0,-1,0])

print LA.norm(a1-a2, 1)
print LA.norm(a1-a2, 2)
print

print LA.norm(b1-b2, np.inf)
print np.corrcoef(b1,b2)[0,1]
print

print np.corrcoef(c1,c2)[0,1]


4.0
2.0

3.0
-2.8665835233e-17

0.0


## Question2 (25 pts)
### Pelase refer lab6 to answer this question.
Consider the training example in following table for a binary decision tree problem.

(a) Computer gini index for the overall collection of training examples.

(b) Based on gini index, which attribute is better, Gender, Car Type, or Shirt Size?

(c) Computer gini index for the Customer ID attribute. Does this attribute have lowest gini index comparing with other three attrabutes? If yes, explain we should not choose Customer ID to split data.

In [5]:
df = pd.DataFrame({"Customer ID" : [i for i in range(1,21)],
                   "Gender" : map(lambda x: "M" if x==1 else "F", [1,1,1,1,1,1,2,2,2,2,1,1,1,1,2,2,2,2,2,2]),
                   "Car Type" : map(lambda x: "Family" if x==1 else ("Sports" if x==2 else "Luxury"), 
                                    [1,2,2,2,2,2,2,2,2,3,1,1,1,3,3,3,3,3,3,3]),
                   "Shirt Size" : ["Small","Medium","Medium","Large","Extra Large","Extra Large","Small","Small",
                                   "Medium","Large","Large","Extra Large","Medium","Extra Large","Small","Small",
                                   "Medium","Medium","Medium","Large"],
                   "Class": map(lambda x: "C0" if x<10 else "C1", [i for i in range(20)])})
df = df[["Customer ID","Gender","Car Type","Shirt Size", "Class"]]

df

,Customer ID,Gender,Car Type,Shirt Size,Class
0,1,M,Family,Small,C0
1,2,M,Sports,Medium,C0
2,3,M,Sports,Medium,C0
3,4,M,Sports,Large,C0
4,5,M,Sports,Extra Large,C0
5,6,M,Sports,Extra Large,C0
6,7,F,Sports,Small,C0
7,8,F,Sports,Small,C0
8,9,F,Sports,Medium,C0
9,10,F,Luxury,Large,C0


## Question 3 (25 pts)
### Please refer lab7 to answer this question.

<img src="Q3.png" alt="Drawing" style="width: 550px;"/>

(a) Which attribute, X, Y, Z, is best feature at the root of decision tree based on classification error?

(b) Build a two-level decision tree. 

In [7]:

# Decision tree.
class Node(object):
    def __init__(self, position="", feature="", split=0, data=[], target=[], left=[], right=[]):
        self.position=position # root, left, or right
        self.feature=feature # Name of the best feature. Use this feature to split current node.
        self.split=split # Best split point of the best feature. 
        self.data=data # Data of features.
        self.target=target # Last column. 
        self.left = left # Left node.
        self.right = right # Right node.

    def __str__(self, level=0):
        ret = ""
        if len(set(self.target))>1:
            ret = "\t"*level+repr(self.position+": "+self.feature+" <= "+str(self.split))+"\n"
        else:
            ret = "\t"*level+repr(self.position+": "+self.target[0])+"\n"
        
        if self.left: ret += self.left.__str__(level+1)
        if self.right: ret += self.right.__str__(level+1)
        return ret

    def __repr__(self):
        return '<tree node representation>'
    
class DecisionTree(object):
    
    def __init__(self, data, target, method, feature_name_, feature_type_):
        self.tree=Node(position="root",data=data, target=target)
        self.method=method
        self.feature_name = feature_name_
        self.feature_type = feature_type_

    # step 1: How to calculate impurity of one node? Please use these two methods: gini and entropy.
    def impurity_node(self,node):
        if len(node)==0:
            return 0
        classes = list(set(node))
        if self.method == "gini":
            gini = 1
            for i in range(len(classes)):
                p = (node==classes[i]).sum() / float(len(node))
                gini -= p**2
            return gini
        elif self.method == "entropy":
            entropy = 0
            for i in range(len(classes)):
                p = (node==classes[i]).sum() / float(len(node))
                if p > 0:
                    entropy += -p*math.log(p)
            return entropy   
        elif self.method == 'error':
            p = []
            for i in range(len(classes)):
                p.append((node==classes[i]).sum() / float(len(node)))
            return 1-max(p)

    # step 2: How to combine Gini index of two children?
    def impurity_children(self, leftnode, rightnode):
        leftgini = self.impurity_node(leftnode)
        rightgini = self.impurity_node(rightnode)
        leftnum = len(leftnode)
        rightnum = len (rightnode)
        allnum = float(leftnum + rightnum)
        gini = leftnum/allnum * leftgini + rightnum/allnum * rightgini
        return gini

    # step 3: How to calculate value of Gain?
    def gain(self, parent, leftnode, rightnode):
        return self.impurity_node(parent)-self.impurity_children(leftnode, rightnode)

    # step 4: Given data and target, return the index of best feature and the best split point of this feature.
    def bestsplit(self, data, target):
        allfeaturegain = []
        allfeaturesplit = []
        for j in range(np.size(data,1)):
            feature_gain = []
            feature = data[:,j]
            
            if self.feature_type[j] == 'numerical':
                feature_sorted = sorted(feature)
                splitpoints = [(feature_sorted[i]+feature_sorted[i+1])/2 for i in range(len(feature_sorted)-1)]
                splitpoints.insert(0,feature_sorted[0]-10)
                splitpoints.append(feature_sorted[-1]+10)
                for i in range(len(splitpoints)):
                    leftnode = target[feature <= splitpoints[i]]
                    rightnode = target[feature > splitpoints[i]]
                    feature_gain.append(self.gain(target, leftnode, rightnode))
                    
                allfeaturegain.append(max(feature_gain))
                allfeaturesplit.append(splitpoints[feature_gain.index(max(feature_gain))])
                
            elif self.feature_type[j] == 'categorical':
                feature_set = list(set(feature))
                for i in range(len(feature_set)):
                    leftnode = target[feature == feature_set[i]]
                    rightnode = target[feature != feature_set[i]]
                    feature_gain.append(self.gain(target, leftnode, rightnode))
                #print feature_gain
                allfeaturegain.append(max(feature_gain))
                allfeaturesplit.append(feature_set[feature_gain.index(max(feature_gain))])   
            
            # Question 2.b, 2.c, 3.a
            print self.feature_name[j]
            print "gain value: ", allfeaturegain[j]
            print "split point: ", allfeaturesplit[j]
            print

        index = allfeaturegain.index(max(allfeaturegain))
        split = allfeaturesplit[index]
        return index, split
    
    # Recursively build the decision tree.
    def nodesplit(self, node, level):
        data = node.data
        target = node.target

        if level <= self.level:
            # Call bestsplit() to calculate best feature and best split point.
            index, split = self.bestsplit(data, target)

            # Update node.
            node.feature = self.feature_name[index]
            node.split = split

            # Split data based on best feature and best split point.
            leftindex = data[:,index]<=split
            rightindex = data[:,index]>split
            lefttarget = target[leftindex]
            righttarget = target[rightindex]

            # Recursively call nodesplit() if child node (1) is not empty  and (2) has more than one classes.
            node.left = Node()
            node.left.position="left"
            node.left.data = data[leftindex, :]
            node.left.target = lefttarget
            if len(set(lefttarget)) > 1:    
                self.nodesplit(node.left, level+1)

            node.right = Node()
            node.right.position="right"
            node.right.data = data[rightindex, :]
            node.right.target = righttarget
            if len(set(righttarget)) > 1:            
                self.nodesplit(node.right, level+1)
                
            #print level, len(node.left.target), len(node.right.target)
            
    def build(self, level):
        self.level = level
        self.nodesplit(self.tree, 1)
        
    def printTree(self):
        print self.tree
    
    def overall(self):
        print self.impurity_node(self.tree.target)
    

######################################

data=df.iloc[:,:-1].as_matrix()
target=df.iloc[:,-1].as_matrix()
tree = DecisionTree(data=data, target=target, method="gini", \
                    feature_name_=df.columns.values, feature_type_=['numerical','categorical','categorical','categorical'])

print "Question 2.a: "
tree.overall()
print 

print "Question 2.b, 2.c: "
tree.build(1)
print


######################################

inputdata = [[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]]
inputtarget = [[5,0,10,45,10,25,5,0],[40,15,5,0,5,0,20,15]]

data,target = [],[]
for i in range(len(inputdata)):
    C1, C2 = inputtarget[0][i], inputtarget[1][i] 
    row = inputdata[i]
    data.extend([row for i in range(C1)])
    data.extend([row for i in range(C2)])
    target.extend('C1' for i in range(C1))
    target.extend('C2' for i in range(C2))
    
tree = DecisionTree(data=np.array(data), target=np.array(target), method="error", \
                    feature_name_=['X','Y','Z'], feature_type_=['categorical','categorical','categorical'])

print "Question 3.a: "
tree.build(1)
print 

print "Question 3.b: "
tree.build(2)
tree.printTree()
print



Question 2.a: 
0.5

Question 2.b, 2.c: 
Customer ID
gain value:  0.5
split point:  10

Gender
gain value:  0.02
split point:  M

Car Type
gain value:  0.333333333333
split point:  Sports

Shirt Size
gain value:  0.00666666666667
split point:  Small


Question 3.a: 
X
gain value:  0.0
split point:  0

Y
gain value:  0.1
split point:  0

Z
gain value:  0.2
split point:  0


Question 3.b: 
X
gain value:  0.0
split point:  0

Y
gain value:  0.1
split point:  0

Z
gain value:  0.2
split point:  0

X
gain value:  0.0
split point:  0

Y
gain value:  0.0
split point:  0

Z
gain value:  0.0
split point:  0

X
gain value:  0.0
split point:  0

Y
gain value:  0.0
split point:  0

Z
gain value:  0.0
split point:  1

'root: Z <= 0'
	'left: X <= 0'
		'left:  <= 0'
		'right:  <= 0'
	'right: X <= 0'
		'left:  <= 0'
		'right:  <= 0'




## Question 4 (25 pts)

### I will distribute paper of question 4 in class. This question does not require you to write code. 